In [6]:
# Imports
import requests
import pandas as pd

In [17]:
# Global Variables
api_key = "RGAPI-29c88dfd-14f0-4192-b087-b6f8cb0a15fa"
base_url = "https://americas.api.riotgames.com"
tag_line = "NA1"

In [27]:
# Function to get unique PUUID by summoner name
def get_puuid_by_name(gameName, tag_line):
    url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tag_line}"
    response = requests.get(url, headers={"X-Riot-Token": api_key})
    puuid = response.json()["puuid"]
    return puuid

# Function to get match history by PUUID
def get_matches_by_puuid(puuid):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    response = requests.get(url, headers={"X-Riot-Token": api_key})
    matches = response.json()
    return matches

# Function to get match data by match ID
def get_match_data_by_match_id(match_id):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
    response = requests.get(url, headers={"X-Riot-Token": api_key})
    match_data = response.json()
    return match_data


In [29]:
print(get_matches_by_puuid(get_puuid_by_name("TS Calamitas", tag_line)))

['NA1_5241920635', 'NA1_5241919970', 'NA1_5241913547', 'NA1_5241913794', 'NA1_5231385593', 'NA1_5231372186', 'NA1_5231322289', 'NA1_5231301748', 'NA1_5231275021', 'NA1_5200311953', 'NA1_5200278399', 'NA1_5200241164', 'NA1_5199932347', 'NA1_5199887190', 'NA1_5194599263', 'NA1_5194337284', 'NA1_5194311268', 'NA1_5193945478', 'NA1_5193937565', 'NA1_5193886687']
